In [1]:
#Librerie utili nel programma
import agentpy as ap
import numpy as np
import random as rd
import math as mp
import matplotlib.pyplot as plt
import matplotlib as mpl
import IPython
import collections
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
import csv

Il programma riportato in questo Notebook, è realizzato per simulare lo starlings flocking, tipico di molti uccelli.
Viene considerato come modello basilare il modello classico Boid, successivamente vengono aggiunte le formule per aggiornare le velocità di ogni singolo uccello ad ogni istante di tempo utilizzando le formule del modello di Cucker - Smale.

In [2]:
#Variabili utili nel programma
H = 1
beta = 0.5

In [3]:
#Normalize a vector to length 1.
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

In [4]:
def Aij(xi,xj):
    return H / (pow(1+pow(np.linalg.norm(xi-xj),2),beta))

In [37]:
#Classe per aggiornare le velocità come d'articolo e di conseguenza anche le posizioni

class Boid(ap.Agent):
    
    def setup(self):
        initvel=np.zeros(3)
        initvel[0] = rd.randint(1,99)/20
        initvel[1] = rd.randint(1,99)/20
        initvel[2] = rd.randint(1,99)/20
        self.velocity=initvel
        
    def setup_pos(self, space):
        self.space = space
        self.neighbors = space.neighbors
        initposition=np.zeros(3)
        initposition[0] = rd.randint(499,599)
        initposition[1] = rd.randint(499,599)
        initposition[2] = rd.randint(499,599)
        space.positions[self]=initposition
        self.pos = space.positions[self]
    
    def update_velocity(self):
        
        global u
        global vel2D
        global pos2D
        global vel3D
        global pos3D
        global step
        pos = self.pos
        ndim = self.p.ndim
        
        # Rule 1 - Formula generale
        nbs = self.neighbors(self, distance=self.p.population)
        nbs_len = len(nbs)
        nbs_pos_array = np.array(nbs.pos)
        nbs_vec_array = np.array(nbs.velocity)
        
        somma=0
            
        for j in range (nbs_len):
            somma += Aij(pos,nbs_pos_array[j])*(nbs_vec_array[j]-self.velocity)
          
        v1=somma
      
        # Rule 5 - Borders
        v5 = np.zeros(ndim)
        d5 = self.p.border_distance
        s5 = self.p.border_strength
        for i in range(ndim):
            if pos[i] < d5:
                v5[i] += s5
            elif pos[i] > self.space.shape[i] - d5:
                v5[i] -= s5
       
        # Update velocity
        self.velocity += v1 + v5
        #self.velocity = normalize(self.velocity)
        
        if self.p.ndim == 2:
            vel2D[step][u]=self.velocity
            pos2D[step][u]=pos
        else:
            vel3D[step][u]=self.velocity
            pos3D[step][u]=pos  
        u+=1
        
        if u==self.p.population:
            u=0
            step+=1
            print(step)
    def update_position(self):

        self.space.move_by(self, self.velocity)

In [38]:
#Inizializzazione della classe del modello Boid
class BoidsModel(ap.Model):

    def setup(self):
        """ Initializes the agents and network of the model. """
        self.space = ap.Space(self, shape=[self.p.size]*self.p.ndim)
        self.agents = ap.AgentList(self, self.p.population, Boid)
        self.space.add_agents(self.agents, random=True)
        self.agents.setup_pos(self.space)

    def step(self):
        """ Defines the models' events per simulation step. """
        self.agents.update_velocity()  # Adjust direction
        self.agents.update_position()  # Move into new direction

In [39]:
#Funzioni per la visualizzazione delle immagini 2D e 3D
def animation_plot_single(m, ax):
    ndim = m.p.ndim
    ax.set_title(f"Boids Flocking Model {ndim}D t={m.t}")
    pos = m.space.positions.values()
    pos = np.array(list(pos)).T  # Transform
    ax.scatter(*pos, s=10, c='black')
    ax.set_xlim(0, m.p.size)
    ax.set_ylim(0, m.p.size)
    if ndim == 3:
        ax.set_zlim(0, m.p.size)
        ax.set_zlabel('z')
        ax.set_zticklabels([])
    ax.grid()
    #ax.set_yticklabels([])
    #ax.set_xticklabels([])
    ax.set_ylabel('y')
    ax.set_xlabel('x')
def animation_plot(m, p):
    projection = '3d' if p['ndim'] == 3 else None
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111, projection=projection)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    mpl.rcParams['animation.embed_limit'] = 100
    return IPython.display.HTML(animation.to_jshtml(fps=20))

Per visualizzare al meglio, sia in 2D e sia in 3D, il fenomeno starlings flocking vengono eseguiti 600 steps per 100 uccelli.

In [40]:
#Dichiaro parametri utili per la computazione 2D
parameters2D = {
    'size': 1000,
    'seed': 123,
    'steps': 300,
    'ndim': 2,
    'population': 100,
    'inner_radius': 3,
    'outer_radius': 10,
    'border_distance': 5,
    'border_strength': 0.5
}

In [41]:
#Computazione 3D con aggiornamenti parametri 2D e calcolo matrici velocità e posizioni 3D
new_parameters = {
    'ndim': 3,
    'population': 100,
    'wall_avoidance_distance': 5,
    'wall_avoidance_strength': 0.3
}

parameters3D = dict(parameters2D)
parameters3D.update(new_parameters)
l=0
l1=0
vicini=0
posizioni=0
spazio=0
step=0 #Tiene traccia numero step
u=0 #Tiene traccia numero uccelli
pos3D=np.zeros((parameters3D['steps'],parameters3D['population'],parameters3D['ndim']))
vel3D=np.zeros((parameters3D['steps'],parameters3D['population'],parameters3D['ndim']))
velinit=np.zeros((parameters3D['population'],parameters3D['ndim']))
posinit=np.zeros((parameters3D['population'],parameters3D['ndim']))
animation_plot(BoidsModel, parameters3D)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141


<ipython-input-37-61bdb4c917d6>:42: RuntimeWarning: overflow encountered in add
  somma += Aij(pos,nbs_pos_array[j])*(nbs_vec_array[j]-self.velocity)


142


<ipython-input-37-61bdb4c917d6>:42: RuntimeWarning: invalid value encountered in subtract
  somma += Aij(pos,nbs_pos_array[j])*(nbs_vec_array[j]-self.velocity)


143


ValueError: list.remove(x): x not in list